╔══<i><b>Alai-DeepLearning</b></i>════════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 10. CNN Architectures**
# Section 3. Transfer Learning을 통한 VGG19 학습하기

### _Objective_

1. 이전 Section에서 학습한 VGG 11을 통해 VGG 19를 학습시키도록 하겠습니다.
  
╚═════════════════════════════════════════╝

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import tensorflow as tf
import cv2
import os
import shutil

!pip install tensorboardcolab
import tensorboardcolab

## Graph Visualization

In [0]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

## 예제 데이터셋) CIFAR-100

In [0]:
from tensorflow.keras.datasets.cifar100 import load_data
(train_x, train_y), (test_x, test_y) = load_data()

In [0]:
class Dataset:
    # 데이터셋을 배치 단위로 처리할 수 있도록 도와주는 Class
    #fix me #

In [0]:
# 이미지 시각화하기
train_set = Dataset(train_x,train_y)
test_set = Dataset(test_x, test_y)
num_sample = 5

sample_x, sample_y = train_set.next_batch(num_sample)

fig = plt.figure(figsize=(10,3))
axes = fig.subplots(1,num_sample)

for ax, image, label in zip(axes, sample_x, sample_y):
    ax.set_title(label)
    ax.imshow(image)
plt.show()

<br>
<br>

# \[ 1. VGG 19 학습하기 \]
---
---

> *VGG Network가 당시에 최고의 성능을 낼 수 있었던 이유는 바로 Layer의 깊이에 있습니다.*<br>
> *VGG Network는 Scratch부터 학습하는 방식이 아닌, 우선 적은 층으로 학습한 후, Transfer Learning을 통해, 깊게 학습시키는 방식으로 진행되었습니다*<br>

<br>

## 1. Network 구성하기
----

![Imgur](https://i.imgur.com/YeoF2AE.png)

* ImageNet에서의 Hyper-Parameter를 바로 적용하기에 몇가지 수정이 필요합니다.<br>
* 이미지의 크기가 imagenet에 비해 8배정도 작기 때문에, 첫 두개의 max-pooling layer를 생략하였습니다.<br>
* Class 수가 1000개에서 100개로 줄었기 때문에, Fully Connected Layer의 노드 수를 4096에서 1024로 줄였습니다.
* 이미지의 복잡도도 좀 더 작다고 판단되기 때문에, Convolution Filter의 갯수들을 모두 1/2로 줄였습니다.

In [0]:
input_shape = (None,32,32,3) # Cifar-100 이미지 크기
num_classes = 100 # cifar-100 클래스 수

graph = tf.Graph()
with graph.as_default():
    images = #fix me#
    labels = #fix me#
    is_train = #fix me#
    lr = #fix me#
    
    with tf.variable_scope("preprocess"):
        vgg_mean = #fix me#
        x = #fix me#
    
    with tf.variable_scope('VGGBlock-1'):
        #fix me #
    # 이미지가 224에 비해 매우 작기 때문에, maxpooling Layer를 생략하였습니다.
        
    with tf.variable_scope('VGGBlock-2'):
        #fix me #
        
    # 이미지가 224에 비해 매우 작기 때문에, maxpooling Layer를 생략하였습니다.            

    with tf.variable_scope('VGGBlock-3'):
        #fix me #
        
        
    with tf.variable_scope('VGGBlock-4'):
        #fix me #
        
                
    with tf.variable_scope('VGGBlock-5'):
        #fix me #
    with tf.variable_scope('FC'):
        #fix me #
        
    
    weight_decay = 5e-4
    with tf.variable_scope("losses"):
        sce = #fix me #
        l2_loss = #fix me #
        loss = #fix me #
    loss = tf.identity(loss, name='loss')    
    
    momentum = 0.9
    with tf.variable_scope("optimizer"):
        global_step = #fix me #
        train_op = #fix me #
    with tf.variable_scope('metrics'):
        top_5, top_5_op = tf.metrics.mean(#fix me #)
        top_1, top_1_op = tf.metrics.mean(#fix me #)
        metric_loss, loss_op = tf.metrics.mean(loss)
    
    # Total과 Count를 reset하는 연산자
    metric_init_op = tf.group(#fix me #)
    # Total과 Count를 갱신하는 연산자
    metric_update_op = tf.group(#fix me #)

    top_5 = tf.identity(top_5, name='top5_acc')
    top_1 = tf.identity(top_1, name='top1_acc')
    metric_loss = tf.identity(metric_loss, name='metric_loss')            
    
    tf.summary.scalar('top5_accuracy', top_5)
    tf.summary.scalar('top1_accuracy', top_1)
    tf.summary.scalar('losses', metric_loss)    
    merged = tf.summary.merge_all()

In [0]:
show_graph(graph)

<br>

## 2. Weight Transfer 적용하기
---

### (1) weights가 저장된 구글 드라이브와 연동하기

In [0]:
# 구글 드라이브랑 연동하기
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# vgg 폴더를 만들어 모델 가져오기
!mkdir vgg/
!cp gdrive/My\ Drive/vgg/* vgg/

### (2) 필요한 weight들을 지정하기

In [0]:
b1_ws = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                             "(VGGBlock-1/conv1)")
b2_ws = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                             "(VGGBlock-2/conv1)")
b3_ws = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                             "VGGBlock-3/(conv1|conv2)")
fc_ws = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                             "FC/")

transfer_weights = b1_ws + b2_ws + b3_ws + fc_ws
transfer_weights

### (3) 필요한 Weight들을 저장한 ckpt로부터 불러오기

학습한 모델은 아래와 같이 가져올 수 있습니다.

In [0]:
with graph.as_default():
    # 가중치 초기화
    sess = tf.Session(graph=graph)
    saver = tf.train.Saver(var_list=transfer_weights)
    saver.restore(sess, "./vgg/vgg11")

### (4) 나머지 Weight들을 초기화 하기

In [0]:
with graph.as_default():
    with tf.variable_scope('initialization'):
        # 모든 variable 가져옴        
        global_vars = tf.global_variables()

        # Variable이 초기화되었는지 안되었는지 확인
        is_not_initialized = sess.run(
            [tf.is_variable_initialized(var) for var in global_vars])

        # Variable 중 초기화되지 않은것만 가져옴
        not_initialized_vars = [v for (v, f) in
                                zip(global_vars, is_not_initialized) if not f]

        # 초기화되지 않은 것이 있으면, 초기화시킴
        if len(not_initialized_vars):
            sess.run(tf.variables_initializer(not_initialized_vars))

위의 과정이 까다롭다면, 먼저 모델을 초기화한 후, Transfer weight들을 덮어씌우는 방식으로도<br>
진행할 수 있습니다.

In [0]:
with graph.as_default():
    # 가중치 초기화
    sess = tf.Session(graph=graph)
    sess.run([tf.global_variables_initializer(),
              tf.local_variables_initializer()])
    saver = tf.train.Saver(var_list=transfer_weights)
    saver.restore(sess, "./vgg/vgg11")

### (5) Tensorboard 세팅하기

In [0]:
# LOG PATH Setting
LOG_DIR = "./log"
if os.path.exists(LOG_DIR):
    shutil.rmtree(LOG_DIR)
os.makedirs(LOG_DIR,exist_ok=True)

tbc = tensorboardcolab.TensorBoardColab(graph_path=LOG_DIR)

train_writer = tf.summary.FileWriter(LOG_DIR+"/train", graph)
test_writer = tf.summary.FileWriter(LOG_DIR+"/test")

### (6) 모델 학습시키기

In [0]:
num_epoch = 20
num_batch = 128
num_data = len(train_set)

for epoch in range(#fix me#):
    # Fitting Model  
    for step in tqdm(range(#fix me#)):
        batch_x, batch_y = train_set.next_batch(num_batch)
        sess.run(train_op,#fix me #)
    # 데이터 셋 섞어주기
    train_set.shuffle()
    
    # Summarize training process
    sess.run(metric_init_op)
    for step in range(0,len(train_set)//1000):
        # 1000개씩 나누어서 metric을 계산합니다.
        batch_x = #fix me#
        batch_y = #fix me#
        sess.run(#fix me#)
    
    summary, top_1_value,top_5_value, loss_value = sess.run(#fix me#)
    
    print("[{:3d} epoch] TRAIN TOP-1 ACC : {:2.2f}% | TOP-5 ACC : {:2.2f}% | LOSS : {:.3f}"
          .format(epoch, top_1_value,top_5_value,loss_value))
    train_writer.add_summary(summary, global_step.eval(sess))

    sess.run(metric_init_op)
    for step in range(len(test_set)//1000):
        batch_x = #fix me #
        batch_y = #fix me #
        sess.run(#fix me #)
        
    summary, top_1_value,top_5_value, loss_value = sess.run(#fix me #)
    print("[{:3d} epoch] TEST TOP-1 ACC : {:2.2f}% | TOP-5 ACC : {:2.2f}% | LOSS : {:.3f}"
          .format(epoch, top_1_value,top_5_value,loss_value))
    test_writer.add_summary(summary, global_step.eval(sess))

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.<br>
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/05/07

---